In [1]:
import pandas as pd
df = pd.concat([
    pd.read_csv(filepath_or_buffer='../input/meterology/casual.csv'),
    pd.read_csv(filepath_or_buffer='../input/meterology/member.csv')
])

In [2]:
df.sample(5)

,ride_id,member_casual,startdate,starttime,stopdate,stoptime,rideable_type,tripduration,max_temperature_c,min_temperature_c,temperature_morning_c,temperature_noon_c,temperature_evening_c,temperature_night_c,precip_total_day_mm,windspeed_max_kmh,cloudcover_avg_percent
996311,B9C526B7E0440D5C,casual,2022-06-25,08:44:28,2022-06-25,08:51:11,electric_bike,6,27,22,22,23,27,22,0.2,17,25.500
298883,30D9CB67F6EEFCD3,member,2023-05-31,14:26:02,2023-05-31,14:47:38,electric_bike,21,25,16,17,24,21,16,1.3,14,53.625
247651,0EE3EF9ACC55DC97,casual,2023-05-23,19:51:51,2023-05-23,20:03:32,classic_bike,11,23,16,16,20,21,16,0.0,9,13.375
1205020,09992615D22746CC,casual,2022-06-21,13:05:31,2022-06-21,15:05:34,classic_bike,120,35,20,20,33,32,21,0.0,23,3.000
1682235,671DEF7F1624C443,member,2022-08-24,13:50:44,2022-08-24,14:07:23,classic_bike,16,30,19,19,26,25,20,0.0,9,2.750


In [3]:
df.shape

(3877902, 17)